#  Utilização do modelo GPT-2 para criação de textos jurídicos com GPUs

Criado com adaptação a textos jurídicos em português por Wilfredo Pacheco (wilfredo.enrique@gmail.com)

Créditos de criação do GPT-2-simple para [Max Woolf](http://minimaxir.com)

*Última atualização: 17/04/2020*

---

Treine a rede neural de criação de textos a partir de qualquer arquivo de texto **de graça usando GPUs do Google Colaboratory** e a biblioteca `gpt-2-simple`, criada por 

Para mais informações sobre a biblioteca `gpt-2-simple`, visite o seguinte repositório: (https://github.com/minimaxir/gpt-2-simple).


Para começar:

1. Copie esse notebook para seu Google Drive para salvas os resultados e suas alterações;
2. Execute as células de código abaixo.


#Baixar o modelo pré-treinado

O modelo abaixo foi treinado a partir do uso das seguintes obras jurídicas:


1.  MENDES, Gilmar Ferreira; BRANCO, Paulo Gustavo Gonet. **Curso de Direito Constitucional.** 6ª ed. São Paulo: Saraiva, 2011.

2. FERNANDES, António Monteiro. **Direito do Trabalho.** Almedina, 2014.

3. BOLZAN, Fabrício (Coord.); LENZA, Pedro. **Direito do Consumidor**: Parte Material, Parte Administrativa. 2013.

4. HUSEK, Carlos Roberto. **Curso de Direito Internacional Público.** LTr, 2012.

5. BRITO, Edvaldo. **Direito tributário e constituição:** estudos e pareceres. Editora Atlas, 2016.

6. TARTUCE, Flávio. **O Novo CPC e o Direito Civil:** Impactos,diálogos e interações. 2015.

7. MELIÁ, Manuel Cancio. **Direito Penal do Inimigo**: Noções-e-críticas. Livraria do Advogado Editora, 2012.

8. CARVALHO, Paulo Barros de. **Curso de Direito Tributário.** Saraiva, 2012.

9. LENZA, Pedro. **Direito Constitucional esquematizado.** São Paulo: Saraiva, 2016.

10. PISCITELLI, Tathiane. **Direito Financeiro Esquematizado.** São Paulo: Editora Método, 2000.

11. GRECO, Rogério. **Curso de Direito Penal.** Vol. 1, 2 e 3. Niterói: Impetus, 2017.

12. RAMOS, André Luiz Santa Cruz. **Direito empresarial esquematizado.** São Paulo: Método, 2014.

**A íntegra das obras não estão presentes no modelo. O que se tem é apenas o treinamento, em linguagem matemática, dos parâmetros matemáticos em matrizes numéricas, não sendo possível a sua leitura em coerência humana.**

O arquivo demorou cerca de 1 semana para ser treinado a partir dos recursos de GPU disponibilizados pelo Google Colaboratory, na maior parte com a GPU NVidia Tesla P100.

O modelo do GPT-2 utilizado foi o intermediário (355M).

A célula de código abaixo baixa o modelo e salva na pasta `checkpoint`.




In [0]:
!gdown -O "JurLib.zip" "https://drive.google.com/uc?export=download&id=1xyPCTmb7SnodVhpEjCRqxQTfVZgu38uX"
!unzip JurLib.zip

Downloading...
From: https://drive.google.com/uc?export=download&id=1xyPCTmb7SnodVhpEjCRqxQTfVZgu38uX
To: /content/JurLib.zip
1.32GB [00:12, 103MB/s]
Archive:  JurLib.zip
  inflating: checkpoint/JurLib/counter  
  inflating: checkpoint/JurLib/model-96000.index  
  inflating: checkpoint/JurLib/checkpoint  
  inflating: checkpoint/JurLib/hparams.json  
  inflating: checkpoint/JurLib/vocab.bpe  
  inflating: checkpoint/JurLib/encoder.json  
  inflating: checkpoint/JurLib/model-96000.meta  
  inflating: checkpoint/JurLib/model-96000.data-00000-of-00001  


## GPU

O Google Colaboratory usa as placas de processamento gráfico (GPU) Tesla P100, Nvidia K80 ou Nvidia T4.

A T4 é ligeiramente melhor e mais rápida do que as K80 para o treinamento do modelo GPT-2, possuindo maior memória e permitindoo treinamento de modelos maiores do GPT-2.

A célula abaixo exige a GPU que foi alocada na presente sessão.

In [0]:
!nvidia-smi

Sat Apr 18 21:05:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Essa célula importará as bibliotecas necessárias para a execução do código

%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



A próxima célula carrega o modelo pré-treinado para utilização.

Caso queira executar mais de uma vez esse código, deverá **resetar a máquina virtual** (reset runtime).

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='JurLib')

Loading checkpoint checkpoint/JurLib/model-96000
INFO:tensorflow:Restoring parameters from checkpoint/JurLib/model-96000


## Gerar texto a partir do modelo pré-treinado

Depois que o modelo foi treinado e carregado na máquina virtual (checkpoint), é possível gerar textos. A função `generate` cria um trecho do tamanho fixado na váriável `batch`.

Other optional-but-helpful parameters for gpt2.generate and friends:

`length`: Número de tokens a serem gerados.

`temperature`: Quanto maior a temperatura, mais randômico, criativo ou incoerente fica o texto. (padrão 0.7, recomendamos, para textos jurídicos, 0.3)

`top_k`: Limita as predições ao top_k de alternativas. Limits the generated guesses to the top k guesses (padrão 0 desabilita essa limitação; se a predição for muito incoerente, definir top_k=40)

`include_prefix`: Define o texto inicial para iniciar a predição de tokens.


In [0]:
gpt2.generate(sess, run_name='JurLib',prefix='O significado da hermenêutica',length=200,nsamples=5,temperature=0.3,top_k=200)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
O significado da hermenêutica jurídica é a base congêneremos ao estudo do fenômeno jurídico, porque assim o entendemos. 
O antecedente é a regra jurídica, o art. 5o, da Constituição da República, pelo qual as decisões administrativas de natureza normativa abstrata (do tributo) vão ter o condão de revogar as leis infraconstitucionais, proclamando-as, declarando-as, emitindo-as. 
O sucede linhas anteriores faz retornar à linha de raciocínio, reescrevendo a tese de que o ato de promulgação do Decreto-lei no 491/1969 é anulá
O significado da hermenêutica jurídica, como objeto de estudo do Direito, está no reconhecimento de que as normas jurídicas cumprem seu papel delimitador, estruturado no plano da tributação, a gratidão, a não cumulatividade, a capacidade contributiva e quem esteja devidamente identificado na formulação das respectivas categorias jurídicas. 
A sua origem está no reconhecimento 

A célula abaixo gera diversos textos com variações diversas nos parâmetros e com diversas frases de cunho jurídico.

In [0]:
frase=["A igualdade de gênero ","O princípio do juiz natural ","O direito do consumidor consiste em "]
for f in frase: 
  for temp in range(2,3):
    for k in range(150,251,50):
      temp2 = temp/10
      print ("k="+str(k)+" temp="+str(temp2))
      gpt2.generate(sess, run_name='JurLib',prefix=f,length=200,nsamples=5,temperature=temp2,top_k=k)

k=50 temp=0.1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
A igualdade de gênero 
A ideia de justiça, que pode ser caracterizada como a ideia de justiça, tem origens mais remota na sociedade internacional. 
A primeira na divisão do Estado é constituída pela tentativa de limitar o poder e, posteriormente, a tentativa de concretizar o direito. 
O Estado brasileiro é quem se infeliz ou quiçá um atrás. A perda de nacionalidade, o deslocamento de órgãos, por exemplo, representa perda de sua competência. 
Ocorre o tema da interferência entre o poder e o indivíduo. 
O Estado brasileiro, por sua vez, não se confunde com
A igualdade de gênero 
A diferença entre o trabalho forçado e o trabalho imposto, no Direito do Trabalho, está no grau de disponibilidade que se opera entre as duas modalidades de situações, e que pode corresponder a um mínimo de ardilosamente recalcitrantes. 
Assim, o trabalho forçado tem que ser prestado em período noventa diár

## Baixando o modelo GPT-2

É necessário baixar as versões do modelo GPT-2 para realizar o seu treinamento ou a geração de texto.

Seguem abaixo as três versões do GPT-2, variando o tamanho, número de parâmetros e complexidade da rede neural:

* `124M` (padrão): o modelo, ocupando 500MB de espaço em disco.
* `355M`: o modelo de tamanho médio, ocupando 1.5GB em disco.
* `774M`: o modelo grande, não sendo atualmente possível realizar o ajuste fino (finetuning) através da maioria das placas do Google Colaboratory, mas é possível realizar a criação de textos a partir de modelos pré-treinados.
* `1558M`: o modelo extra-grande, que foi efetivamente usado nos melhores resultados obtidos pelo OpenAI. Não irá funcionar com as GPUs K80 e P100. (Assim como o modelo `774M`, não é possível realizar finetuning).

Modelos maiores conseguem ser treinados e perceber maiores complexidades textuais, mas demoram mais para passarem pelo processo de finetuning e demoram mais tempo para gerar textos. É possível alterar a versão do GPT-2 a partir da mudança da variável `model_name` nas células abaixo.

A próxima célula irá baixar a versão escolhida e salvar na atual sessão de execução do Google Colab. Importante lembrar que o conteúdo não permanecerá salvo de forma persistente, devendo ser baixado a cada nova sessão da máquina virtual.

Dessa forma, esse passo deve ser feito novamente se a máquina virtual for resetada às configurações originais (factory reset runtime) ou se decorrido muito tempo do início de sua execução. Porém, será preservado em caso de runtime reset.


In [0]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 528Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 99.1Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 724Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:11, 126Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 438Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 111Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 196Mit/s]                                                       


If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

## Ajuste fino (Finetune) do GPT-2

A próxima célula realizará o ajuste fino do GPT-2. O objetivo é criar uma sessão persistente do TensorFlow que armazenará o treinamento dos parâmetros das redes neurais e realizando o treino a cada passo (steps). Caso queira deixar o treinamento de forma permanente, defina 'steps = -1'.

O Google Colaboratory normalmente reseta em 4 horas, então, recomenda-se o treinamento de até 4.000 steps por sessão.

O resultado do treinamento é salvo na pasta **checkpoint**, cujo caminho absoluto é '**/content/checkpoint**'.

Os arquivos que armazenam os parâmetros salvos (checkpoint) ficam armazenados na pasta '/checkpoint/JurLib' por padrão, e são salvos a cada 500 passos da mesma sessão (o que pode se ajustado na variável save_every=500) e quando o código da célula é finalizado.

Cada sessão de treino pode levar cerca de 4 horas, então é importante que salve o andamento do treino em sua conta do Google Drive.

**IMPORTANTE:** Caso queira executar novamente essa célula dentro dessa sessão, é necessário que **resete a máquina virtual antes**, (Runtime -> Restart Runtime). Você precisará rodar a primeira célula de definição de importações de bibliotecas (imports), mas não será necessário baixar os arquivos novamente, tendo em vista que não apagados. 

Segue abaixo explicação sobre outros parâmetros para `finetuning`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=4000,
              restore_from='latest',
              run_name='JurLib',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/JurLib/model-92000
INFO:tensorflow:Restoring parameters from checkpoint/JurLib/model-92000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:36<00:00, 36.73s/it]


dataset has 9184741 tokens
Training...
Saving checkpoint/JurLib/model-92000
======== SAMPLE 1 ========
), o Estado não se responsabiliza pelos atos ilícitos praticados, senão antes da decretação dos terminos. Nesse caso, o sujeito deveria pleitear ao banco outro o recebimento da importância arbitrada. 
Este quebramento pode acontecer a partir da decretação de intervenção federal, em se tratando de território federal, de acordo com o art. 85 da CLT: “um sujeito a decisão do Banco Central do Brasil, no curso do mandato, pode impugnar o Tesouro Nacional e os demais Banco Central do Brasil porque lhes deu o pronunciamento”.24 
¦ 9.13.3.6. Inofiĺcio do prazo prescricional das obras em relação às suas executas: aplicação da Lei n. 8.625, de 18.07.1993 (Lei do Império), segundo a qual o executado poderá alegar, em 12 meses, a ocorrência de crédito com destinação específica, ainda que para bens móveis, e aplicando-se o art. 27 da Lei n. 8.625, de 18.07.1993, que regula os contribuintes no PROC

#Salvar treinamento
O código abaixo tem a finalidade de mover a pasta `checkpoint` para o Google Drive, com o objetivo de não perder o treinamento quando houver a reinicialização da máquina virtual.

In [0]:
!cp -avr "/content/checkpoint/" "/content/drive/My Drive/checkpointJurLib/"
!cp -avr "/content/samples/" "/content/drive/My Drive/GPT-samplesJurLib"

'/content/checkpoint/JurLib/model-88000.index' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.index'
'/content/checkpoint/JurLib/model-96000.meta' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-96000.meta'
'/content/checkpoint/JurLib/model-88000.data-00000-of-00001' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.data-00000-of-00001'
'/content/checkpoint/JurLib/model-88000.meta' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.meta'
'/content/checkpoint/JurLib/events.out.tfevents.1587067170.917c34133274' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/events.out.tfevents.1587067170.917c34133274'
'/content/checkpoint/JurLib/hparams.json' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/hparams.json'
'/content/checkpoint/JurLib/encoder.json' -> '/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/encoder.json'
'/content/checkpoint/JurLib/vocab.bpe' ->

#Carregar o modelo salvo no Google Drive
O objetivo da célula a seguir é carregar o arquivo de treinamento novamente na máquina virtual

In [0]:
!mkdir checkpoint
!cp -avr "/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib" "/content/checkpoint/JurLib"

'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib' -> '/content/checkpoint/JurLib'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/.ipynb_checkpoints' -> '/content/checkpoint/JurLib/.ipynb_checkpoints'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.index' -> '/content/checkpoint/JurLib/model-88000.index'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.data-00000-of-00001' -> '/content/checkpoint/JurLib/model-88000.data-00000-of-00001'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-88000.meta' -> '/content/checkpoint/JurLib/model-88000.meta'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/events.out.tfevents.1587067170.917c34133274' -> '/content/checkpoint/JurLib/events.out.tfevents.1587067170.917c34133274'
'/content/drive/My Drive/checkpointJurLib/checkpoint/JurLib/model-92000.data-00000-of-00001' -> '/content/checkpoint/JurLib/model-92000.data-00000-of-00001'
'/content/drive/My Drive/

A função abaixo gera uma grande quantidade de texto e salva em arquivo texto, com uma marcação de data e tempo.

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )